## 3.2.1 训练一个分词器

SentencePiece 是由 Google 开发并维护的一个开源工具，它为文本生成系统提供了一个无监督的文本分词器（tokenizer）和去分词器（detokenizer）。SentencePiece 的设计目标是能够直接处理原始文本数据，而不需要进行任何预处理步骤，如去除空格或标点符号等。它的主要特点是能够在字符级别和单词级别之间找到一个平衡，使用所谓的“子词”（subword）分割方法来构建词汇表。

SentencePiece 使用了几种不同的子词分割算法，其中最常用的是 Byte Pair Encoding (BPE) 和 Unigram Language Model (ULM)。参见[2.2 分词器Tokenizer](<2.2 分词器Tokenizer.ipynb>)

使用 SentencePiece 进行文本的分词和去分词主要包括几个步骤：准备数据、训练模型以及应用模型。下面详细介绍这些步骤，并引用相关资料中的细节。

**准备数据**

SentencePiece 的输入数据格式要求每一行文本占据一行，保存在 `.txt` 文件中。每行文本应尽可能保持内容完整，长度不限，确保语义完整即可。例如：

```
你好是一个汉语词语
这是一个测试句子
...
```
下面使用清华大学自然语言处理实验室[THUCTC的语料库](http://thuctc.thunlp.org/)来训练一个分词器。THUCNews是根据新浪新闻RSS订阅频道2005~2011年间的历史数据筛选过滤生成，包含74万篇新闻文档（2.19 GB），均为UTF-8纯文本格式。THUCTC在原始新浪新闻分类体系的基础上，重新整合划分出14个候选分类类别：**财经、彩票、房产、股票、家居、教育、科技、社会、时尚、时政、体育、星座、游戏、娱乐。** 
本次训练选取大约5万篇文章，将选取的文章合并成一个文档corpus.txt进行训练。

**训练模型**

数据文件准备好，就可以开始训练 SentencePiece 模型，训练过程调用 `SentencePieceTrainer.train()` 方法完成。下述例子基于名为 `corpus.txt` 的语料库训练一个 BPE 模型，并生成前缀为 `zh_pu_model` 的模型文件和词汇表文件：

```python
spm.SentencePieceTrainer.train('--input=corpus.txt --model_prefix=zh_pu_model --vocab_size=50000 --model_type=bpe')
```

参数说明：
- `--control_symbols=<foo>,<bar>`: 定义控制符号。
- `--user_defined_symbols=<user1>,<user2>`: 添加用户自定义符号。
- `--input=<input file>`: 输入文本文件路径。
- `--model_prefix=<model file>`: 输出模型文件前缀。
- `--vocab_size=8000`: 词汇表大小。
- `--model_type=char/word/bep/unigram`: 分词类型。
- `--normalization_rule_name=nfkc_cf`: 文本归一化规则。

函数入参也可以通过变量方式传递：

In [ ]:
import sentencepiece as spm

spm.SentencePieceTrainer.train(
    input='corpus.txt',       # 中文语料文件
    model_prefix='zh_pu_model',  # 输出模型前缀
    model_type = 'bpe',
    vocab_size=50000,        # 词汇表大小
    character_coverage=0.9995,
    pad_id=0,                # 特殊token配置
    unk_id=1,
    bos_id=2,
    eos_id=3,
    user_defined_symbols=['[CLS]', '[SEP]', '[MASK]', '<|endoftext|>']
)


print("分词器训练完成")

领域知识可以在现有模型基础上继续训练：

In [ ]:
spm.SentencePieceTrainer.train(
    input=domain_corpus.txt,
    model_prefix='domain_adapted',
    input_format="text",
    pretokenized_delimiter="",
    train_extremely_large_corpus=True,
    input_sentence_size=1000000,
    seed_sentencepiece_size=1000000,
    shrink_vocab=True,
    num_sub_iterations=2,
    model_file="zh_sp_model.model"  # 加载已有模型
)

**使用模型**

训练完成后，就可以加载模型进行文本的编码（分词）和解码（去分词）。以下代码是加载并使用之前训练好的模型的一个示例，利用 SentencePiece 对文本进行编码和解码的过程。首先，创建一个 `SentencePieceProcessor` 实例，并通过 `load()` 方法加载之前训练得到的模型。然后，可以使用 `encode_as_pieces()` 或 `encode_as_ids()` 方法对文本进行编码，分别返回子词片段或对应的 ID 序列；同样地，可以使用 `decode_pieces()` 或 `decode_ids()` 方法将 ID 序列还原成原始文本。

In [ ]:
import sentencepiece as spm

sp = spm.SentencePieceProcessor()
sp.load('zh_sp_model.model') 

# 编码：将文本转换为 ID 序列
encoded_pieces = sp.encode_as_pieces('努力是一种习惯。')
encoded_ids = sp.encode_as_ids('努力是一种习惯。')
print(encoded_pieces)
print(encoded_ids)

# 解码：将 ID 序列还原为文本
decoded_text = sp.decode_pieces(['▁', '努力', '是一种', '习惯', '。'])
decoded_text_from_ids = sp.decode_ids([46252, 772, 2079, 999, 46253])
print(decoded_text_from_ids)

## 3.2.2 数据加载与模型训练
### 加载数据与分词

- TextDataset在初始化过程中，对整个文本进行编码得到token ids列表。然后，基于seq_len和stride参数，通过滑动窗口的方式从token ids中提取多个样本，每个样本由两部分组成：input_ids和target_ids。input_ids是当前窗口中的token序列，而target_ids是向后移一位的相同序列，用于预测下一个token。
- 使用torch的数据加载器DataLoader读取文件，DataLoader提供了批量加载、打乱数据等功能，便于模型训练时使用。
- 使用前面训练的ChineseTokenizer对文本进行token化处理，此处也可以使用现有的词汇表，如BertTokernizer
- 数据文件分为训练数据和验证数据，可以从原始数据中分割一部分作为验证数据

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from chinese_tokenizer import ChineseTokenizer

class TextDataset(Dataset):
    def __init__(self, txt, tokenizer, seq_len, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)
        for i in range(0, len(token_ids) - seq_len, stride):
            input_chunk = token_ids[i:i + seq_len]
            target_chunk = token_ids[i + 1: i + seq_len + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader(txt, tokenizer, batch_size=4, seq_len=256, stride=16, shuffle=True, drop_last=True, num_workers=0):
    dataset = TextDataset(txt, tokenizer, seq_len, stride)
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader


with open("train_file.txt", "r", encoding="utf-8") as f:
    train_text = f.read()

tokenizer = ChineseTokenizer("zh_sp_model.model")
print(tokenizer.vocab_size,)

dataloader = create_dataloader(
    train_text,
    tokenizer,
    batch_size=4,
    seq_len=16,
    stride=16
)    

### 批量数据的损失计算
- logits的矩阵形状为[batch_size, seq_len, vocab_size]，每一个tokenID在logits中都有一个**next token**的预测概率值，训练就是要最大化这个概率值。
<img src='./images/cal_batch_loss.png' alt="The Pile" height='400' >

- 下面的测试程序演示了批量数据的loss计算，假设batch_size大小为8，维度为32，随机生成输入数据和目标数据。

In [ ]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits, _ = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), target_batch.view(-1))
    return loss

model = testModel({
    "vocab_size": 30000,  # 重要！需匹配分词器词汇量
    "dim": 768,
    "n_heads": 8,
    "n_layers": 6,
    "mlp_dim": 512,
    "n_kv_heads": 4,
    "dropout": 0.1
}).to(device)

input_batch = torch.randint(low=0, high=100, size=(8,32))
target_batch = torch.randint(low=0, high=100, size=(8,32))

loss = calc_loss_batch(input_batch, target_batch, model, device)

print(loss) # tensor(10.4201, device='cuda:0', grad_fn=<NllLossBackward0>)

### 训练过程
- 训练过程
<center>
<img src='./images/train_proc.png' alt="训练过程" height='400' >
</center>

In [ ]:
def train(
    model,
    tokenizer,
    train_loader,
    val_loader,
    epochs: int = 10,
    lr: float = 3e-4,
    eval_freq: int = 100,
    save_dir: str = "checkpoints",
    device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
):
   
    optimizer = AdamW(model.parameters(), lr=lr)   

    train_losses = []
    val_losses = []    

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        
        for step, (inputs, targets) in enumerate(progress_bar):        
            loss = calc_loss_batch(inputs, targets, model, device)           

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()          
            if step % eval_freq == 0:
                avg_train_loss = total_loss / (step + 1)
                val_loss = evaluate(model, val_loader, device)
                
                train_losses.append(avg_train_loss)
                val_losses.append(val_loss)
                
                progress_bar.set_postfix({
                    "train_loss": avg_train_loss,
                    "val_loss": val_loss
                })
                print(f"Ep {epoch+1} (Step {step:06d}): " f"Train loss {avg_train_loss:.3f}, Val loss {val_loss:.3f}")
        print("\nGenerating sample text:")
        sample_text = generate_text(
            model,
            tokenizer,
            prompt="中国的首都是 ",
            max_len=100,
            temperature=0.7,
            top_k=50,
        )
        print(sample_text)

用少量数据验证一下程序的运行结果：
<center>
<img src='./images/trained_result.png' alt="训练结果" height='400' >
</center>